In [1]:
from overlap_functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os
from scipy.optimize import fsolve

In [2]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
def equation(n):
    N_mu = np.log(n) * np.log(n)
    return n * N_mu**2 - T


mu_max = 6
T_list = np.array([5.4E5, 7.7E5, 1.21E6])
n_list = []
for T in T_list:
    n_initial_guess = 300
    n = int(np.round(fsolve(equation, n_initial_guess)[0]))
    print('n = ', n, ', N_mu = ', mu_max * np.log(n), ', for T =', T)
    print('mu_max =', np.log(n))
    n_list.append(n)

n_list = np.array(n_list)
N_mu_list = np.round(mu_max * np.log(n_list))

alpha = 1 + 0.5j
n_precision = 1000

print('n_list =', n_list)
print('N_mu_list =', N_mu_list)
print('T_list =', T_list)

n =  411 , N_mu =  36.1115592869774 , for T = 540000.0
mu_max = 6.018593214496234
n =  510 , N_mu =  37.40646435431023 , for T = 770000.0
mu_max = 6.234410725718371
n =  673 , N_mu =  39.070471977868365 , for T = 1210000.0
mu_max = 6.511745329644728
n_list = [411 510 673]
N_mu_list = [36. 37. 39.]
T_list = [ 540000.  770000. 1210000.]


In [ ]:
trial_times = 1

MSE_list = []
MSE_noisy_list = []
MSE_correction_list = []

TD_list = []
TD_noisy_list = []
TD_correction_list = []


for n, N_mu in zip(n_list, N_mu_list):
    

    N_mu = int(N_mu)
    mu_list = np.linspace(-mu_max, mu_max, N_mu)
    nu_list = np.linspace(-mu_max, mu_max, N_mu)


    perfect_phi_grid = np.zeros((N_mu, N_mu), dtype=complex)
    for idx_mu, mu in enumerate(mu_list):
        for idx_nu, nu in enumerate(nu_list):
            perfect_phi_grid[idx_mu, idx_nu] = compute_perfect_phi(mu, nu, alpha, t=1)
            perfect_phi_grid = np.where(np.isnan(perfect_phi_grid), 1, perfect_phi_grid)
    # integral1 = np.trapz(perfect_phi_grid * np.conjugate(perfect_phi_grid), x=nu_list, axis=1)
    # integral1 = 1 / 2 / np.pi * np.trapz(integral1, x=mu_list) 
    integral1 = 1   

    for _ in range(trial_times):
        estimated_phi_grid = np.zeros((N_mu, N_mu), dtype=complex)
        estimated_phi_grid_noisy = np.zeros((N_mu, N_mu), dtype=complex)
        estimated_phi_grid_correction = np.zeros((N_mu, N_mu), dtype=complex)
        for idx_mu, mu in enumerate(tqdm(mu_list)):
            for idx_nu, nu in enumerate(nu_list):
                
                L_limit = find_suitable_L(mu, nu, alpha)
                X_precision = np.linspace(-L_limit, L_limit, n_precision)
                pdf = normalized_cat_state_pdf(mu, nu, alpha, X_precision)

                try:
                    kappa = 1
                    X_samples = samplings_with_noisy_pdf(pdf, X_precision, n, kappa)
                    W_Gaussian = reconstruct_pdf_mle(X_samples, X_precision)
                    phi_t1_vals_gaussian, optimal_bd = compute_gaussian_phi_at_t_optimal(mu, nu, alpha, X_precision, X_samples, W_Gaussian, t = 1)
                    estimated_phi_grid[idx_mu, idx_nu] = phi_t1_vals_gaussian
                except:
                    estimated_phi_grid[idx_mu, idx_nu] = np.nan

                try:
                    kappa = 0.85
                    X_samples = samplings_with_noisy_pdf(pdf, X_precision, n, kappa)
                    W_Gaussian = reconstruct_pdf_mle(X_samples, X_precision)
                    phi_t1_vals_gaussian, optimal_bd = compute_gaussian_phi_at_t_optimal(mu, nu, alpha, X_precision, X_samples, W_Gaussian, t = 1)
                    estimated_phi_grid_noisy[idx_mu, idx_nu] = phi_t1_vals_gaussian

                    phi_t1_vals_gaussian, optimal_bd = compute_gaussian_phi_at_t_optimal(mu, nu, alpha, X_precision, X_samples, W_Gaussian, t = 1 / kappa)
                    phi_t1_vals_gaussian *= 1 / np.exp( - ((1 - kappa) / kappa)**2 / 2   )
                    estimated_phi_grid_correction[idx_mu, idx_nu] = phi_t1_vals_gaussian
                except:
                    estimated_phi_grid_noisy[idx_mu, idx_nu] = np.nan
                    estimated_phi_grid_correction[idx_mu, idx_nu] = np.nan

        estimated_phi_grid = np.where(np.isnan(estimated_phi_grid), 1, estimated_phi_grid)
        estimated_phi_grid_noisy = np.where(np.isnan(estimated_phi_grid_noisy), 1, estimated_phi_grid_noisy)
        estimated_phi_grid_correction = np.where(np.isnan(estimated_phi_grid_correction), 1, estimated_phi_grid_correction)
        
        integral2 = np.trapz(perfect_phi_grid * np.conjugate(estimated_phi_grid), x=nu_list, axis=1)
        integral2 = 1 / 2 / np.pi * np.trapz(integral2, x=mu_list)
        MSE = (np.abs(integral1 - integral2))**2
        MSE_list.append(MSE)
        TD_list.append(np.sqrt(np.abs(1 - integral2)))

        integral2 = np.trapz(perfect_phi_grid * np.conjugate(estimated_phi_grid_noisy), x=nu_list, axis=1)
        integral2 = 1 / 2 / np.pi * np.trapz(integral2, x=mu_list)
        MSE = (np.abs(integral1 - integral2))**2
        MSE_noisy_list.append(MSE)
        TD_noisy_list.append(np.sqrt(np.abs(1 - integral2)))

        integral2 = np.trapz(perfect_phi_grid * np.conjugate(estimated_phi_grid_correction), x=nu_list, axis=1)
        integral2 = 1 / 2 / np.pi * np.trapz(integral2, x=mu_list)
        MSE = (np.abs(integral1 - integral2))**2
        MSE_correction_list.append(MSE)
        TD_correction_list.append(np.sqrt(np.abs(1 - integral2)))

    print('CAT estimate: ', 'n =', n, 'error =', np.mean(MSE_list), 'TD =', np.mean(TD_list))
    print('CAT noisy: ', 'n =', n, 'error =', np.mean(MSE_noisy_list), 'TD =', np.mean(TD_noisy_list))
    print('CAT correction: ', 'n =', n, 'error =', np.mean(MSE_correction_list), 'TD =', np.mean(TD_correction_list))



100%|██████████| 36/36 [00:41<00:00,  1.16s/it]


CAT estimate:  n = 411 error = 3.37370446149688e-05 TD = 0.07621259666404336
CAT noisy:  n = 411 error = 0.012466024943038351 TD = 0.33414271564183107
CAT correction:  n = 411 error = 9.159319428922953e-07 TD = 0.0309361169810201


 32%|███▏      | 12/37 [00:14<00:32,  1.30s/it]